In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
dataset=pd.read_excel('crimes_total.xlsx')

In [ ]:
dataset.columns

Index(['Year', 'crimes.total'], dtype='object')

In [ ]:
#Converting the columns to list
year_list=dataset["Year"].tolist()
crimes_list=dataset["crimes.total"].tolist()

In [ ]:
!pip install wget

import wget
wget.download('https://raw.githubusercontent.com/BorisMuzellec/MissingDataOT/master/utils.py')

import numpy as np
import pandas as pd
from utils import *
import torch
import seaborn as sns

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=ca1f09f481b7af6001fd8f5fd31ee8d9228e39b39a6f1f94d64e657a58ca3fb7
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
np.random.seed(0)
def produce_NA(X, p_miss, mecha="MCAR", opt=None, p_obs=None, q=None):
    """
    Generate missing values for specifics missing-data mechanism and proportion of missing values.

    Parameters
    ----------
    X : torch.DoubleTensor or np.ndarray, shape (n, d)
        Data for which missing values will be simulated.
        If a numpy array is provided, it will be converted to a pytorch tensor.
    p_miss : float
        Proportion of missing values to generate for variables which will have missing values.
    mecha : str,
            Indicates the missing-data mechanism to be used. "MCAR" by default, "MAR", "MNAR" or "MNARsmask"
    opt: str,
         For mecha = "MNAR", it indicates how the missing-data mechanism is generated: using a logistic regression ("logistic"), quantile censorship ("quantile") or logistic regression for generating a self-masked MNAR mechanism ("selfmasked").
    p_obs : float
            If mecha = "MAR", or mecha = "MNAR" with opt = "logistic" or "quanti", proportion of variables with *no* missing values that will be used for the logistic masking model.
    q : float
        If mecha = "MNAR" and opt = "quanti", quantile level at which the cuts should occur.

    Returns
    ----------
    A dictionnary containing:
    'X_init': the initial data matrix.
    'X_incomp': the data with the generated missing values.
    'mask': a matrix indexing the generated missing values.s
    """

    to_torch = torch.is_tensor(X) ## output a pytorch tensor, or a numpy array
    if not to_torch:
        X = X.astype(np.float32)
        X = torch.from_numpy(X)

    if mecha == "MAR":
        mask = MAR_mask(X, p_miss, p_obs).double()
    elif mecha == "MNAR" and opt == "logistic":
        mask = MNAR_mask_logistic(X, p_miss, p_obs).double()
    elif mecha == "MNAR" and opt == "quantile":
        mask = MNAR_mask_quantiles(X, p_miss, q, 1-p_obs).double()
    elif mecha == "MNAR" and opt == "selfmasked":
        mask = MNAR_self_mask_logistic(X, p_miss).double()
    else:
        mask = (torch.rand(X.shape) < p_miss).double()

    X_nas = X.clone()
    X_nas[mask.bool()] = np.nan

    return {'X_init': X.double(), 'X_incomp': X_nas.double(), 'mask': mask}

In [ ]:
#the data is corrupted with 18% of generated missing values
X_init=np.array(year_list)
X_miss_mcar = produce_NA(X_init, p_miss=0.15, mecha="MCAR")

year_mcar = X_miss_mcar['X_incomp']
R_mcar = X_miss_mcar['mask']

print("Percentage of generated missing values: ", (R_mcar.sum()).numpy()/np.prod(R_mcar.size())*100, " %")

print(year_mcar)

year_list_15=year_mcar.tolist()

Percentage of generated missing values:  10.606060606060606  %
tensor([  nan, 1951., 1952., 1953., 1954., 1955., 1956.,   nan, 1958., 1959.,
        1960., 1961., 1962., 1963., 1964., 1965., 1966.,   nan, 1968., 1969.,
        1970., 1971., 1972., 1973., 1974., 1975., 1976., 1977., 1978., 1979.,
          nan, 1981., 1982., 1983., 1984., 1985., 1986., 1987.,   nan, 1989.,
        1990., 1991., 1992., 1993., 1994., 1995., 1996., 1997., 1998., 1999.,
        2000., 2001.,   nan, 2003., 2004., 2005., 2006., 2007.,   nan, 2009.,
        2010., 2011., 2012., 2013., 2014., 2015.], dtype=torch.float64)


In [ ]:
#the data is corrupted with 18% of generated missing values
X_init=np.array(crimes_list)
X_miss_mcar = produce_NA(X_init, p_miss=0.15, mecha="MCAR")

crimes_mcar = X_miss_mcar['X_incomp']
R_mcar = X_miss_mcar['mask']

print("Percentage of generated missing values: ", (R_mcar.sum()).numpy()/np.prod(R_mcar.size())*100, " %")

print(crimes_mcar)

#the array is converted back to list
crimes_list_15=crimes_mcar.tolist()

Percentage of generated missing values:  19.696969696969695  %
tensor([ 2784.,  3284.,  3160.,  2909.,    nan,  3357.,  3488.,  3774.,    nan,
         4033.,  3982.,  4108.,  4263.,  4466.,  4799.,  5801.,  6063.,  6421.,
           nan,    nan,    nan,    nan,  8509.,  8055.,  8275.,  9221.,  9720.,
        10233.,  9706.,  9840.,    nan, 11247., 11817., 11515., 11793., 12195.,
           nan, 13020., 12875.,    nan,    nan, 13915.,    nan, 13664., 12670.,
        12982., 13294., 13554., 13344.,    nan, 13694., 13370., 13663., 13997.,
        13885., 13753., 13490., 14280.,    nan, 15101., 14605., 14988., 14734.,
        14603., 14890., 15342.], dtype=torch.float64)


In [ ]:
data_dict={
    'year_15':year_list_15,
    'crimes_15':crimes_list_15

}

crimes_data_missing_15_percent = pd.DataFrame(data_dict)

#the list are merged to form a dataframe

In [ ]:
#the dataframe is converted to excel format
crimes_data_missing_15_percent.to_excel('crimes_15percent_missing.xlsx', index=False)

In [ ]:
#Interpolation technique
#First method: nearest
dataset_15=pd.read_excel('crimes_15percent_missing.xlsx')

dataset_15['year_15'].interpolate(method = 'nearest', inplace = True)
dataset_15['crimes_15'].interpolate(method = 'nearest', inplace = True)

dataset_15.to_excel('dataset_15_nearest_crime.xlsx', index=False)

#second method: pad
dataset_15=pd.read_excel('crimes_15percent_missing.xlsx')

dataset_15['year_15'].interpolate(method = 'pad', inplace = True)
dataset_15['crimes_15'].interpolate(method = 'pad', inplace = True)

dataset_15.to_excel('dataset_15_pad_crime.xlsx', index=False)

#third method: linear

dataset_15=pd.read_excel('crimes_15percent_missing.xlsx')

dataset_15['year_15'].interpolate(method = 'linear', inplace = True)
dataset_15['crimes_15'].interpolate(method = 'linear', inplace = True)

dataset_15.to_excel('dataset_15_linear_crime.xlsx', index=False)

#fourth method: cubicspline

dataset_15=pd.read_excel('crimes_15percent_missing.xlsx')

dataset_15['ip_year_15']= dataset_15['year_15'].interpolate(method = 'cubicspline', inplace = True)
dataset_15['ip_crimes_15']=dataset_15['crimes_15'].interpolate(method = 'cubicspline', inplace = True)

dataset_15.to_excel('dataset_15_cubicspline_crime.xlsx', index=False)

